# Flux sampling

## Basic usage

The easiest way to get started with flux sampling is using the `sample` function in the `flux_analysis` submodule. `sample` takes at least two arguments: a cobra model and the number of samples you want to generate.

In [1]:
from cobra.test import create_test_model
from cobra.flux_analysis import sample

model = create_test_model("textbook")
s = sample(model, 100)
s.head()

,ACALD,ACALDt,ACKr,ACONTa,ACONTb,ACt2r,ADK1,AKGDH,AKGt2r,ALCD2x,...,RPI,SUCCt2_2,SUCCt3,SUCDi,SUCOAS,TALA,THD2,TKT1,TKT2,TPI
0,-1.178850,-0.320232,-0.272449,10.243257,10.243257,-0.272449,2.224921,4.784288,-0.018712,-0.858617,...,-1.972200,11.329566,11.731676,123.124131,-4.784288,1.961696,5.412904,1.961696,1.957472,6.922416
1,-2.416852,-0.259989,-1.135790,10.000593,10.000593,-1.135790,13.128625,6.472998,-0.121168,-2.156864,...,-0.598236,9.007054,9.965535,161.981643,-6.472998,0.577321,1.521955,0.577321,0.568911,9.283538
2,-0.750359,-0.010754,-1.020824,10.497342,10.497342,-1.020824,2.833915,8.235925,-0.536536,-0.739605,...,-0.748569,7.961637,8.325093,309.610201,-8.235925,0.730358,4.876850,0.730358,0.723035,9.238769
3,-1.942355,-0.084652,-0.173589,10.440011,10.440011,-0.173589,0.010040,7.520618,-0.190567,-1.857703,...,-0.838915,7.060465,7.868009,312.437262,-7.520618,0.813028,4.239887,0.813028,0.802618,8.762697
4,-0.071480,-0.012750,-2.035411,11.951297,11.951297,-2.035411,19.086657,9.030340,-0.170290,-0.058730,...,-1.013284,30.209397,30.226259,345.302336,-9.030340,1.003938,44.770608,1.003938,1.000179,7.526600


By default sample uses the `optgp` method based on the [method presented here](http://dx.doi.org/10.1371/journal.pone.0086587) as it is suited for larger models and can run in parallel. By default the sampler uses a single process. This can be changed by using the `processes` argument.

In [2]:
print("One process:")
%time s = sample(model, 1000)
print("Two processes:")
%time s = sample(model, 1000, processes=2)

One process:
CPU times: user 6.54 s, sys: 53.5 ms, total: 6.6 s
Wall time: 6.68 s
Two processes:
CPU times: user 46.3 ms, sys: 20.2 ms, total: 66.5 ms
Wall time: 3.45 s


Alternatively you can also user Artificial Centering Hit-and-Run for sampling by setting the method to `arch`. `arch` does not support parallel execution but has good convergence and is almost markovian.

In [3]:
s = sample(model, 100, method="arch")

In general setting up the sampler is expensive since initial search directions are generated by solving many linear programming problems. Thus, we recommend to generate as many samples as possible in one go. However, this might require finer control over the sampling procedure as described in the following section.

## Advanced usage

### Sampler objects

The sampling process can be controlled on a lower level by using the sampler classes directly.

In [4]:
from cobra.flux_analysis.sampling import OptGPSampler, ARCHSampler

Both sampler classes have standardized interfaces and take some additional argument. For instance the `thinning` factor. "Thinning" means only recording samples every n iterations. A higher thinning factors mean less correlated samples but also larger computation times. By default the samplers use a thinning factor of 100 which creates roughly uncorrelated samples. If you want less samples but better mixing feel free to increase this parameter. If you want to study convergence for your own model you might want to set it to 1 to obtain all iterates.

In [5]:
arch = ARCHSampler(model, thinning=10)

`OptGPSampler` has an additional `processes` argument specifying how many processes are used to create parallel sampling chains. This should be in the order of your CPU cores for maximum efficiency. As noted before class initialization can take up to a few minutes due to generation of initial search directions. Sampling on the other hand is quick.

In [6]:
optgp = OptGPSampler(model, processes=4)

### Sampling and validation

Both samplers have a sample function that generates samples from the intialized object and act like the `sample` function described above, only that this time it will oncly accept a single argument, the number of samples. For `OptGPSampler` the number of samples should be a mutiple of the number of processes, otherwise it will be increased to the nearest multiple automatically.

In [7]:
s1 = arch.sample(100)

s2 = optgp.sample(100)

You can call `sample` repeatedly and both samplers are optimized to generate large amount of samples without falling into "numerical traps". All sampler objects have a `validate` function in order to check if a set of points are feasible and give detailed information about feasibility violations in a form of a short code denoting feasibility. Here the short code is a combination of any of the following letters:

- "v" - valid point
- "l" - lower bound violation
- "u" - upper bound violation
- "e" - equality violation (meaning the point is not a steady state)

For instance for a random flux distribution (should not be feasible):

In [8]:
import numpy as np

bad = np.random.uniform(-1000, 1000, size=len(model.reactions))
arch.validate(np.atleast_2d(bad))

array(['le'], 
      dtype='<U3')

And for our generated samples:

In [9]:
arch.validate(s1)

array(['v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v',
       'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v',
       'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v',
       'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v',
       'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v',
       'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v',
       'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v',
       'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v'], 
      dtype='<U3')

### Batch sampling

Sampler objects are made for generating billions of samples, however using the `sample` function might quickly fill up your RAM when working with genome-scale models. Here, the `batch` method of the sampler objects might come in handy. `batch` takes two arguments, the number of samples in each badge and the number of batches. This will make for sense with a small example. 

Let's assume we want to qunatify what proportion of our samples will grow. For that we might want to generate 10 batches of 50 samples each and measure what percentage of the individual 100 samples show a growth rate larger than 0.1. Finally, we want to calculate the mean and standard deviation of those individual percentages.

In [10]:
# Biomass reaction is reaction number 12
print(model.reactions[12])

Biomass_Ecoli_core


In [11]:
counts = [np.mean(s[:, 12] > 0.1) for s in optgp.batch(100, 10)]
print("Usually {:.2f}% +- {:.2f}% grow...".format(np.mean(counts) * 100.0, np.std(counts) * 100.0))

Usually 6.10% +- 1.14% grow...
